# Request
Show us a visual representation that explores danceability, popularity, and song key. Potential questions that could be asked are: What song key has the most danceability? How does popularity and song key relate? Are the most popular songs also the most danceable?

# In this notebook
- Averages
- Min and max
- Other plots to try and get a better understanding of the data
- Quick conclusions

# Setup
This notebook needs the following Python libraries:
- matplotlib
- pandas
- sqlite3
- numpy
- seaborn
- plotly
- kaleido

In the section after comments is a cell you may run to bring these libraries in named "Configure". 

Note - as I discovered, plotly charts can't be rendered on GitHub. The workaround is to render plotly graphs to png or some other static format.

# Comments
Most of what I've done code wise could be simplified with using dataframes exclusively. I've leveraged SQL mostly because of my experience with it, but I believe much of what I could written, if I had a little more time, could be refactored and simplified. 

This was my first exposure to leveraging seaborn and matplotlib, so I enjoyed the opportunity to learn about these as I went along with this assignment. Much of what was done here I learned from Youtube and observing other work on Kaggle.com, and could also be simplified or enhanced (adding color graphs/heat maps to tables for example) if I could spend more time reviewing the documentation.

The graphs and tables in this notebook were done on a wide-screen monitor - some of the code may need to be adjusted to view on other resolutions.


# Configure
Uncomment and run the following section only if you don't have the libaries already installed.

In [1]:
# %pip install matplotlib
# %pip install pandas
# %pip install numpy
# %pip install seaborn
# %pip install plotly
# note - sqlite3 should be included by default, otherwise un-comment and run the line below
# %pip install pysqlite
# %pip install kaleido



### Averages

#### Approach

 - Load data from "music_genre.csv" in folder using pandas. 
 - Save the data frame back to SQLite for convenience. 
    - Note that when we pull data out of sqlite we're normalizing popularity to make it easier to chart (trying to get the value between 0 and 1).
 - Plot items using pyplot from matlab and others.
 - Output to tables, coloring max/min values if possible. Alternatively, be explicit about min/max values, etc.

In [2]:
import matplotlib.pyplot as plt
import plotly.express as ex
import plotly.graph_objs as go
import pandas as pd
import sqlite3
import numpy as np
from plotly.subplots import make_subplots

# read csv
data = pd.read_csv('music_genre.csv')

# sort the data by key
# I would like to revisit this approach as there may be a better way to do this without duplicating the data
plot_data = data.sort_values(by='key')

# create an in-memory sqlite database and save the data frame to it
# used to using sql, although we can use sql against dataframes as well
conn = sqlite3.connect(":memory:")
data.to_sql(name='music', con=conn)

# get the average by key
# we multiply the popularity by 0.01 to try and normalize (produce a value between 0 and 1)
query = """SELECT 
                key,
                ROUND(AVG(danceability), 3) avg_dance, 
                ROUND(AVG(popularity) * 0.01, 3) avg_pop
            FROM 
                music
            WHERE
                danceability IS NOT NULL
            AND
                popularity IS NOT NULL                 
            GROUP BY 
                key"""

data_avg = pd.read_sql(query, conn)


In [3]:
# split these into two different data frames for plotting in table
data_avg_dance = data_avg.drop(columns='avg_pop', axis=1)
data_avg_pop = data_avg.drop(columns='avg_dance', axis=1)

color_dance = ['rgb(40,40,40)', ['rgb(255,0,0)' if v == data_avg['avg_dance'].min(
) or v == data_avg['avg_dance'].max() else 'rgb(10,10,10)' for v in data_avg['avg_dance'].tolist()]]
color_pop = ['rgb(40,40,40)', ['rgb(255,0,0)' if v == data_avg['avg_pop'].min(
) or v == data_avg['avg_pop'].max() else 'rgb(10,10,10)' for v in data_avg['avg_pop'].tolist()]]

# https://community.plotly.com/t/conditional-formatting-for-table/27877/2

# make two plots, 1 row, but with 2 columns, both which will have tables
fig = make_subplots(rows=1, cols=2, specs=[
                    [{"type": "table"}, {"type": "table"}]])

# plot the first table for danceability
table_dance = go.Table(
    header=dict(
        values=['Key', 'Average Danceability'],
        line=dict(color='rgb(50,50,50)'),
        align=['left']*2,
        font=dict(color=['rgb(45,45,45)']*2, size=12),
    ),
    cells=dict(values=data_avg_dance.T.values,
               line=dict(color='#506784'),
               align=['left']*5,
               font=dict(family="Arial", size=12, color=color_dance),
               format=[None, ",.3f"],
               fill=dict(color='rgb(245,245,245)'))
)

# plot the second table for popularity
table_pop = go.Table(
    header=dict(
        values=['Key', 'Average Popularity'],
        line=dict(color='rgb(50,50,50)'),
        align=['left']*2,
        font=dict(color=['rgb(45,45,45)']*2, size=12),
    ),
    cells=dict(values=data_avg_pop.T.values,
               line=dict(color='#506784'),
               align=['left']*5,
               font=dict(family="Arial", size=12, color=color_pop),
               format=[None, ",.3f"],
               fill=dict(color='rgb(245,245,245)'))
)

fig = fig.add_trace(table_dance, row=1, col=1)
fig = fig.add_trace(table_pop, row=1, col=2)
fig.show("png")
# fig.write_html("avg_table.html",include_plotlyjs='cdn')

# output some quick observations
print("Observations about Danceability:")
print('The minimum average dance value is: ', str(data_avg['avg_dance'].min(
)), 'by key ', data_avg.loc[data_avg['avg_dance'] == data_avg['avg_dance'].min()]['key'].to_string(index=False))
print('The maximum average dance value is: ', str(data_avg['avg_dance'].max(
)), 'by key ', data_avg.loc[data_avg['avg_dance'] == data_avg['avg_dance'].max()]['key'].to_string(index=False))
print("Observations about Popularity:")
print('The minimum average pop value is: ', str(data_avg['avg_pop'].min(
)), 'by key ', data_avg.loc[data_avg['avg_pop'] == data_avg['avg_pop'].min()]['key'].to_string(index=False))
print('The maximum average pop value is: ', str(data_avg['avg_pop'].max(
)), 'by key ', data_avg.loc[data_avg['avg_pop'] == data_avg['avg_pop'].max()]['key'].to_string(index=False))

# plot both against each other
# https://datascienceparichay.com/article/plot-a-line-chart-in-python-with-matplotlib/
# not sure if a line graph is the best visual here, because we're not doing changes over time
plt.plot(data_avg['key'].tolist(), data_avg['avg_pop'].tolist())
plt.plot(data_avg['key'].tolist(), data_avg['avg_dance'].tolist())
plt.xlabel("Key")
plt.ylabel("Avg Danceability and Popularity")
plt.legend(["Pop", "Dance"])
plt.show()


#### Quick observation
It appears the max average danceability and popularity are the highest for songs in the key C#, and lowest for the key of D#



#### Quick Sanity Check
Use data frame to also view averages -

In [ ]:
# sanity checks against SQL
print('danceability averages by key')
print(data.groupby('key')['danceability'].mean())
print('popularity averages by key')
print(data.groupby('key')['popularity'].mean())


It appears that our code seems to match what data frames is also reporting.

### Min / Max

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# grab danceability, popularity, and key from the table we just created in SQL
# we likely can also do this using dataframes exclusively
query = """
        SELECT 
            key,
            ROUND(MIN(danceability), 3) min_dance, 
            ROUND(MAX(danceability), 3) max_dance, 
            ROUND(MIN(popularity), 3) min_pop, 
            ROUND(MAX(popularity) * 0.01, 3) max_pop
        FROM 
            music
        WHERE
            danceability IS NOT NULL
        AND
            popularity IS NOT NULL                     
        GROUP BY 
            key"""

# print(pd.read_sql_query(query, conn))
df = pd.read_sql(query, conn)

# https://community.plotly.com/t/conditional-formatting-for-table/27877
# appears to be a bug in the color formatting if you specify a height, doesn't color the correct cell
color_min_max = [
    'rgb(40,40,40)',
    ['rgb(255,0,0)' if a == df['min_dance'].min() or a == df['min_dance'].max(
    ) else 'rgb(10,10,10)' for a in df['min_dance'].tolist()],
    ['rgb(255,0,0)' if b == df['max_dance'].min() or b == df['max_dance'].max(
    ) else 'rgb(10,10,10)' for b in df['max_dance'].tolist()],
    ['rgb(255,0,0)' if c == df['min_pop'].min() or c == df['min_pop'].max()
     else 'rgb(10,10,10)' for c in df['min_pop'].tolist()],
    ['rgb(255,0,0)' if d == df['max_pop'].min() or d == df['max_pop'].max()
     else 'rgb(10,10,10)' for d in df['max_pop'].tolist()]
]

table_min_max = go.Table(
    header=dict(values=df.columns,
                line=dict(color='rgb(50,50,50)'),
                align=['left']*2,
                font=dict(color=['rgb(45,45,45)']*2, size=12),
                ),
    cells=dict(values=df.T.values,
               line=dict(color='#506784'),
               align=['left']*5,
               font=dict(family="Arial", size=12, color=color_min_max),
               format=[None, ",.3f"],
               fill=dict(color='rgb(245,245,245)'))
)

fig = go.Figure(data=table_min_max)
fig.show("png")
#fig.write_html("minmax_table.html",include_plotlyjs='cdn')


# https://www.statology.org/matplotlib-table/
#fig, ax = plt.subplots()
# fig.patch.set_visible(False)
# ax.axis('off')
# ax.axis('tight')

#table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
# table.set_fontsize(20)
#table.scale(2, 2)
# plt.show()

plt.plot(df['key'].tolist(), df['min_dance'].tolist())
plt.plot(df['key'].tolist(), df['max_dance'].tolist())
plt.plot(df['key'].tolist(), df['max_pop'].tolist())
plt.xlabel("Key")
plt.ylabel("Min, Max Dance and Max Pop")
plt.legend(["Min Dance", "Max Dance", "Max Pop"])
plt.show()


#### Quick Observation
It appears the highest popular song is in the key of B. The lowest popular song is in the key of F#.

It's not easy to tell from this graph, but it appears the highest dancable song is in the key of G#, while the highest lowest dancable song is in the key of D#. 

It also appears to be consistent that regardless of key, you can always have a song in a key that isn't very danceable.

### Other Plots

Plots as reviewed from the original [dataset](https://www.kaggle.com/datasets/vicsuperman/prediction-of-music-genre) on Kaggle. Example [here](https://www.kaggle.com/code/ayalaco/comprehensive-eda-class-consolidation-and-svm).

I wanted to try and play around with other visuals to get a better sense of the data, so this is that attempt.

##### Popularity

In [ ]:
import seaborn as sns
sns.set(rc={"figure.figsize": (15, 10)})  # width, #height
sns.boxplot(x=plot_data['key'], y=plot_data['popularity'], data=plot_data)


Looking at the above boxplot, it appears that this reinforces that on average, songs in the key of C# have the highest popularity, and on average, the lowest is in the key of D# (looking at the median line).

We also can see that there's an outlier for a song in key of B being the highest popular song in the data.

In [ ]:
sns.scatterplot(x=plot_data['key'], y=plot_data['popularity'], data=plot_data)


The scatter plot above doesn't show much other insights, but it does reinforce that there was an outlier for popularity in the key of B.

In [ ]:
plot_data.groupby('key')['popularity'].plot.kde()
plt.legend()
plt.show()


##### Danceability

In [ ]:
import seaborn as sns
sns.set(rc={"figure.figsize": (15, 10)})  # width, #height
sns.boxplot(x=plot_data['key'], y=plot_data['danceability'], data=plot_data)


This boxplot also reinforces that the most popular songs on average appear to be in the key of C#, and again, the lowest popular songs on average are in the key of D#.

In [ ]:
sns.scatterplot(x=plot_data['key'],
                y=plot_data['danceability'], data=plot_data)


This scatter plot doesn't reveal much other than that there are some outliers scoring low for songs in the key of F#. This is also supported by the previous boxplot.

In [ ]:
plot_data.groupby('key')['danceability'].plot.kde()
plt.legend()
plt.show()


# Initial Conclusions

It would appear that there's a strong correlation between songs that are popular and danceable. It would also appear that if an artist wanted to try and write a popular song that people could dance to, it might make sense to write it in the key of C#.